In [10]:
import openrouteservice as ors
import folium
import folium.plugins as plugins
import random

client = ors.Client(key='5b3ce3597851110001cf6248facc383c642b420fa9818331b313169e')

In [11]:
import pandas as pd
# Read Data
nodes = pd.read_csv("SolutionData\\Data_10_1_1\\Nodes.csv")
truckRoute = pd.read_csv("SolutionData\\Data_10_1_1\\TSPTruckRoute.csv")
droneRoute = pd.read_csv("SolutionData\\Data_10_1_1\\DroneRoute.csv")
nodes

,Index,X,Y,Type
0,0,28.708302,41.028654,TRUCK
1,1,29.278845,40.967896,TRUCK
2,2,29.260502,41.001772,UAVELIGIBLE
3,3,28.763392,41.036689,UAVELIGIBLE
4,4,28.951533,41.024870,UAVELIGIBLE
5,5,29.144979,40.980963,UAVELIGIBLE
6,6,28.899415,40.992468,UAVELIGIBLE
7,7,28.820287,41.039158,UAVELIGIBLE
8,8,28.853122,41.022436,UAVELIGIBLE
9,9,28.674086,41.156142,UAVELIGIBLE


In [12]:
m = folium.Map(location=[40.80533085, 29.442315], tiles='cartodbpositron', zoom_start=13)

# Add Nodes
droneEligableChance = 0.7
for index, row in nodes.iterrows():
    color = None
    if(row["Type"] == "UAVELIGIBLE"):
        color = "orange"
    else:
        color = "cyan"
    if(index == 0 or index == nodes.shape[0]-1):
        icon = plugins.BeautifyIcon(icon = "truck", iconShape="marker", backgroundColor="white")
    else:
        icon = plugins.BeautifyIcon(icon = "", iconShape="marker", number = row["Index"], backgroundColor=color)
    folium.Marker([row["Y"], row["X"]], icon=icon, popup= row["Index"],).add_to(m)
m

In [13]:
truckRoute

,Index,X,Y
0,0,28.708302,41.028654
1,9,28.674086,41.156142
2,2,29.260502,41.001772
3,1,29.278845,40.967896
4,5,29.144979,40.980963
5,4,28.951533,41.024870
6,6,28.899415,40.992468
7,8,28.853122,41.022436
8,7,28.820287,41.039158
9,3,28.763392,41.036689


In [14]:
# Add Truck Routes

for i in range(0, truckRoute.shape[0] - 1):
    popup = "{i} --> {j}"
    
    coordinates = [[truckRoute["X"][i], truckRoute["Y"][i]], [truckRoute["X"][i+1], truckRoute["Y"][i+1]]]
    route = client.directions(
    coordinates=coordinates,
    profile='driving-car',
    format='geojson',
    validate=False,)
    plugins.AntPath(locations=[list(reversed(coord)) 
                           for coord in 
                           route['features'][0]['geometry']['coordinates']], 
                           tooltip = popup.format(i = truckRoute["Index"][i], j = truckRoute["Index"][i+1]), linecap ="round").add_to(m)
   
m

In [7]:
droneRoute

,DroneIndex,iIndex,jIndex,kIndex,iX,iY,jX,jY,kX,kY,Launch[i],Arrival[j],Launch[j],Retrieval[k]
0,0,1,9,10,29.278845,40.967896,28.674086,41.156142,28.708302,41.028654,5296.865000,6843.435000,6903.435000,8326.985000
1,0,6,4,5,28.899415,40.992468,28.951533,41.024870,29.144979,40.980963,2480.975000,2847.795000,2907.795000,4149.005000
2,0,0,7,3,28.708302,41.028654,28.820287,41.039158,28.763392,41.036689,60.000000,714.295000,774.295000,1177.522415
3,0,5,2,1,29.144979,40.980963,29.260502,41.001772,29.278845,40.967896,4239.005000,4789.930000,4849.930000,5206.865000
4,0,3,8,6,28.763392,41.036689,28.853122,41.022436,28.899415,40.992468,1237.522415,1705.652415,1765.652415,2420.975000


In [8]:
# Add Drone Routes
colorsForDroneRoutes = ["red", "purple", "yellow", "green", "orange"]

for rowNumber in range(0, droneRoute.shape[0]):
    i = (droneRoute["iY"][rowNumber], droneRoute["iX"][rowNumber])
    j = (droneRoute["jY"][rowNumber], droneRoute["jX"][rowNumber])
    k = (droneRoute["kY"][rowNumber], droneRoute["kX"][rowNumber])
    
    color = colorsForDroneRoutes[droneRoute["DroneIndex"][rowNumber]]
    
    ijRouteToolTip = "{i} --> {j} | LaunchTime[{i}]: {launch} / ArrivalTime[{j}]: {arrival}"
    jkRouteToolTip = "{j} --> {k} | LaunchTime[{j}]: {launch} / RetrievalTime[{k}]: {retrieval}"
    plugins.AntPath(locations = [i,j], color=color, tooltip = ijRouteToolTip.format(i = droneRoute["iIndex"][rowNumber],
                                                                                   j = droneRoute["jIndex"][rowNumber],
                                                                                   launch = droneRoute["Launch[i]"][rowNumber],
                                                                                   arrival = droneRoute["Arrival[j]"][rowNumber])).add_to(m)
    plugins.AntPath(locations = [j,k], color=color, tooltip = jkRouteToolTip.format(j = droneRoute["jIndex"][rowNumber],
                                                                                   k = droneRoute["kIndex"][rowNumber],
                                                                                   launch = droneRoute["Launch[j]"][rowNumber],
                                                                                   retrieval = droneRoute["Retrieval[k]"][rowNumber])).add_to(m)
m    

In [9]:
m.save('map.html')

In [10]:
route["features"][0]["properties"]["summary"]

{'distance': 45455.3, 'duration': 2600.4}